In [356]:
from typing import List
import numpy as np
import math

In [357]:
def panel_panel_dgemm_a_t(
    rows: int,
    cols: int,
    panel_a_rows: int,
    panel_a_cols: int,
    panel_b_rows: int,
    panel_b_cols: int,
    start_row_a: int,
    start_col_a: int,
    start_row_b: int,
    start_col_b: int,
    write_row: int,
    write_col: int,
    A: List[float],
    B: List[float],
    C: List[float],
) -> None:
    """
    We perform panel-panel multiplication using the values in A and B and write the result to C

    The matrices are assumed to be stored in row-major order in 1D arrays with the exception of A

    `start_row_a` tells us which row of A the panel starts at if we were to view A as a 2D matrix.
    `start_col_a` tells us which column of A the panel starts at if we were to view A as a 2D matrix.
    `panel_a_rows` tells us how many rows the panel has if we were to view A as a 2D matrix.
    `panel_a_cols` tells us how many columns the panel has if we were to view A as a 2D matrix.

    Similarly, `start_row_b`, `start_col_b`, `panel_b_rows`, and `panel_b_cols` tell us the same
    information about B.

    `write_row` tells us which row of C we should start writing to if we were to view C as a 2D matrix.
    `write_col` tells us which column of C we should start writing to if we were to view C as a 2D matrix.
    """
    for i in range(panel_a_cols):
        a_c = start_col_a + i
        b_r = start_row_b + i
        for k in range(panel_a_rows):
            a_r = start_row_a + k
            for j in range(panel_b_cols):
                b_c = start_col_b + j
                a_flat = a_c * cols + a_r
                b_flat = b_r * rows + b_c
                out_flat = (write_row + k) * rows + j + write_col
                C[out_flat] += A[a_flat] * B[b_flat]

In [358]:
def transpose(N, X):
    out = [0] * (N**2)
    for i in range(N):
        for j in range(N):
            original = i*N + j
            tranposed = j*N + i
            out[tranposed] = X[original]
    return out

In [359]:
def panel_panel_dgemm(
    rows: int,
    cols: int,
    panel_a_rows: int,
    panel_a_cols: int,
    panel_b_rows: int,
    panel_b_cols: int,
    start_row_a: int,
    start_col_a: int,
    start_row_b: int,
    start_col_b: int,
    write_row: int,
    write_col: int,
    A: List[float],
    B: List[float],
    C: List[float],
) -> None:
    """
    We perform panel-panel multiplication using the values in A and B and write the result to C.

    The matrices are assumed to be stored in row-major order in 1D arrays

    `start_row_a` tells us which row of A the panel starts at if we were to view A as a 2D matrix.
    `start_col_a` tells us which column of A the panel starts at if we were to view A as a 2D matrix.
    `panel_a_rows` tells us how many rows the panel has if we were to view A as a 2D matrix.
    `panel_a_cols` tells us how many columns the panel has if we were to view A as a 2D matrix.

    Similarly, `start_row_b`, `start_col_b`, `panel_b_rows`, and `panel_b_cols` tell us the same
    information about B.

    `write_row` tells us which row of C we should start writing to if we were to view C as a 2D matrix.
    `write_col` tells us which column of C we should start writing to if we were to view C as a 2D matrix.
    """
    for k in range(panel_a_rows):
        a_row = start_row_a + k
        for i in range(panel_a_cols):
            a_col = start_col_a + i
            b_row = start_row_b + i
            for j in range(panel_b_cols):
                b_col = start_col_b + j
                a_flat = a_row * rows + a_col
                b_flat = b_row * rows + b_col
                c_flat = (write_row + k) * rows + j + write_col
                C[c_flat] += A[a_flat] * B[b_flat]

def panel_panel_dgemm_bt(
    rows: int,
    cols: int,
    panel_a_rows: int,
    panel_a_cols: int,
    panel_b_rows: int,
    panel_b_cols: int,
    start_row_a: int,
    start_col_a: int,
    start_row_b: int,
    start_col_b: int,
    write_row: int,
    write_col: int,
    A: List[float],
    B: List[float],
    C: List[float],
) -> None:
    """
    We perform panel-panel multiplication using the values in A and B and write the result to C

    The matrices are assumed to be stored in row-major order in 1D arrays with the exception of B

    `start_row_a` tells us which row of A the panel starts at if we were to view A as a 2D matrix.
    `start_col_a` tells us which column of A the panel starts at if we were to view A as a 2D matrix.
    `panel_a_rows` tells us how many rows the panel has if we were to view A as a 2D matrix.
    `panel_a_cols` tells us how many columns the panel has if we were to view A as a 2D matrix.

    Similarly, `start_row_b`, `start_col_b`, `panel_b_rows`, and `panel_b_cols` tell us the same
    information about B.

    `write_row` tells us which row of C we should start writing to if we were to view C as a 2D matrix.
    `write_col` tells us which column of C we should start writing to if we were to view C as a 2D matrix.
    """
    for k in range(panel_a_rows):
        a_row = start_row_a + k
        for j in range(panel_b_cols):
            b_col = start_col_b + j
            for i in range(panel_a_cols):
                a_col = start_col_a + i
                b_row = start_row_b + i
                a_flat = a_row * rows + a_col
                b_flat = b_col * rows + b_row
                c_flat = (write_row + k) * rows + j + write_col
                C[c_flat] += A[a_flat] * B[b_flat]

In [360]:
def panel_panel_dgemm_recurse_a_t(
    rows: int,
    cols: int,
    panel_a_rows: int,
    panel_a_cols: int,
    panel_b_rows: int,
    panel_b_cols: int,
    start_row_a: int,
    start_col_a: int,
    start_row_b: int,
    start_col_b: int,
    write_row: int,
    write_col: int,
    A: List[float],
    B: List[float],
    C: List[float],
    block_size: int
) -> None:
    loop_cap = math.ceil(cols / block_size) if block_size < cols else 0 + 1 # remove +1 for C++
    for i in range(loop_cap):
        start_col_a = start_col_a + i * block_size
        start_row_b = start_col_a
        panel_a_cols = min(block_size, cols - start_col_a)
        panel_b_rows = panel_a_cols
        panel_panel_dgemm_a_t(
            rows,
            cols,
            panel_a_rows,
            panel_a_cols,
            panel_b_rows,
            panel_b_cols,
            start_row_a,
            start_col_a,
            start_row_b,
            start_col_b,
            write_row,
            write_col,
            A,
            B,
            C
        )

In [361]:
rows = 4
cols = 4
A = np.array([i for i in range(rows*cols)], dtype=np.float32)
B = A.copy() * -1

In [362]:
A_matrix = A.reshape((rows, cols))
B_matrix = B.reshape((rows, cols))

In [363]:
A_panel_1 = A_matrix[0:2, :]
B_panel_1 = B_matrix[:, 0:2]

In [364]:
A_panel_2 = A_matrix[2:4, :]
B_panel_2 = B_matrix[:, 2:4]

In [365]:
A_panel_1, B_panel_1, A_panel_2, B_panel_2

(array([[0., 1., 2., 3.],
        [4., 5., 6., 7.]], dtype=float32),
 array([[ -0.,  -1.],
        [ -4.,  -5.],
        [ -8.,  -9.],
        [-12., -13.]], dtype=float32),
 array([[ 8.,  9., 10., 11.],
        [12., 13., 14., 15.]], dtype=float32),
 array([[ -2.,  -3.],
        [ -6.,  -7.],
        [-10., -11.],
        [-14., -15.]], dtype=float32))

In [366]:
upper_left = A_panel_1 @ B_panel_1
lower_left = A_panel_2 @ B_panel_1
upper_right = A_panel_1 @ B_panel_2
lower_right = A_panel_2 @ B_panel_2

In [367]:
combined = np.zeros((rows, cols), dtype=np.float32)
combined[0:2, 0:2] = upper_left
combined[2:4, 0:2] = lower_left
combined[0:2, 2:4] = upper_right
combined[2:4, 2:4] = lower_right

In [368]:
combined

array([[ -56.,  -62.,  -68.,  -74.],
       [-152., -174., -196., -218.],
       [-248., -286., -324., -362.],
       [-344., -398., -452., -506.]], dtype=float32)

In [369]:
flat_a = A.flatten().tolist()
flat_b = B.flatten().tolist()
flat_c = [0] * (rows * cols)

In [370]:
flat_b_t = transpose(rows, flat_b)
flat_a_t = transpose(cols, flat_a)

In [371]:
panel_a_cols = cols
panel_b_rows = rows
start_col_a = 0
start_row_b = 0
panel_block_size = 3
loop_cap = math.ceil(rows / panel_block_size) if panel_block_size < rows else 0 + 1 #remove the +1 when C++
print(loop_cap)
for i in range(loop_cap):
    start_row_a = i * panel_block_size
    panel_a_rows = min(panel_block_size, rows - start_row_a)
    for j in range(loop_cap):
        start_col_b = j * panel_block_size
        panel_b_cols = min(panel_block_size, cols - start_col_b)
        write_row = start_row_a
        write_col = start_col_b
        panel_panel_dgemm(
            rows, cols, panel_a_rows, panel_a_cols, panel_b_rows, panel_b_cols, start_row_a, start_col_a, start_row_b, start_col_b, write_row, write_col, flat_a, flat_b, flat_c
        )

2


In [372]:
matrix_c = np.array(flat_c).reshape((rows, cols))

In [373]:
combined, matrix_c

(array([[ -56.,  -62.,  -68.,  -74.],
        [-152., -174., -196., -218.],
        [-248., -286., -324., -362.],
        [-344., -398., -452., -506.]], dtype=float32),
 array([[ -56.,  -62.,  -68.,  -74.],
        [-152., -174., -196., -218.],
        [-248., -286., -324., -362.],
        [-344., -398., -452., -506.]]))

In [374]:
flat_c = [0] * (rows * cols)

In [375]:
panel_a_cols = cols
panel_b_rows = rows
start_col_a = 0
start_row_b = 0
panel_block_size = 3
loop_cap = math.ceil(rows / panel_block_size) if panel_block_size < rows else 0 + 1 #remove the +1 when C++
for i in range(loop_cap):
    start_row_a = i * panel_block_size
    panel_a_rows = min(panel_block_size, rows - start_row_a)
    for j in range(loop_cap):
        start_col_b = j * panel_block_size
        panel_b_cols = min(panel_block_size, cols - start_col_b)
        write_row = start_row_a
        write_col = start_col_b
        panel_panel_dgemm_recurse_a_t(
            rows, cols, panel_a_rows, panel_a_cols, panel_b_rows, panel_b_cols, start_row_a, start_col_a, start_row_b, start_col_b, write_row, write_col, flat_a_t, flat_b, flat_c, 2
        )

outer loop has start_row_a: 0, start_col_a: 0, start_row_b: 0, start_col_b: 0
outer loop has panel_a_rows: 3, panel_a_cols: 4, panel_b_rows: 4, panel_b_cols: 3
start_row_a: 0, start_col_a: 0, start_row_b: 0, start_col_b: 0
panel_a_rows: 3, panel_a_cols: 2, panel_b_rows: 2, panel_b_cols: 3
write_row 0, write_col 0
----------
At 0 adding to 0 from 0.0 * -0.0 = -0.0
At 1 adding to 0 from 0.0 * -1.0 = -0.0
At 2 adding to 0 from 0.0 * -2.0 = -0.0
At 4 adding to 0 from 4.0 * -0.0 = -0.0
At 5 adding to 0 from 4.0 * -1.0 = -4.0
At 6 adding to 0 from 4.0 * -2.0 = -8.0
At 8 adding to 0 from 8.0 * -0.0 = -0.0
At 9 adding to 0 from 8.0 * -1.0 = -8.0
At 10 adding to 0 from 8.0 * -2.0 = -16.0
At 0 adding to 0.0 from 1.0 * -4.0 = -4.0
At 1 adding to 0.0 from 1.0 * -5.0 = -5.0
At 2 adding to 0.0 from 1.0 * -6.0 = -6.0
At 4 adding to 0.0 from 5.0 * -4.0 = -20.0
At 5 adding to -4.0 from 5.0 * -5.0 = -25.0
At 6 adding to -8.0 from 5.0 * -6.0 = -30.0
At 8 adding to 0.0 from 9.0 * -4.0 = -36.0
At 9 adding 

In [376]:
matrix_c = np.array(flat_c).reshape((rows, cols))

In [377]:
combined, matrix_c

(array([[ -56.,  -62.,  -68.,  -74.],
        [-152., -174., -196., -218.],
        [-248., -286., -324., -362.],
        [-344., -398., -452., -506.]], dtype=float32),
 array([[ -56.,  -62.,  -68.,  -74.],
        [-152., -174., -196., -218.],
        [-248., -286., -324., -362.],
        [-344., -398., -452., -506.]]))